In [8]:
import cv2
from ultralytics import YOLO
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision as tv

In [9]:
yolo_model = YOLO("yolov8n.pt")

In [ ]:
def load_midas_model():
    model_type = "DPT_Large"
    model = torch.hub.load("intel-isl/MiDaS", model_type)
    model.eval()
    return model

def get_depth_map(model, image):
    transform = tv.Compose([
        tv.Resize(384, 384),
        tv.ToTensor(),
        tv.Normalize(mean = [0.5], std = [0.5]),
    ]) 
    input_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        depth_map = model(input_tensor)
    depth_map = depth_map.squeeze().numpy()
    return depth_map

In [13]:
midas_model = load_midas_model()

NameError: name 'load_midas_model' is not defined

In [10]:
def detect_objects(frame):
    detection_info = []
    #carica l'immagine
    img = cv2.imread(frame)

    #esegui il rilevamento con yolov8
    yolo_results = yolo_model(img)

    #ottieni la mappa di profondita' con MiDaS
    depth_map = get_depth_map(midas_model, frame)

    #ottieni i risultati del rilevamento (results[0] indica le informazioni ottenute dall'unica immagine in input, se ci fossero piu di una immagine allora sarebbe results[i] con i = 1...n)
    detection = yolo_results[0].boxes.xyxy #coordinate della bounding box
    confidences = yolo_results[0].boxes.conf #confidenza della rilevazione
    class_ids = yolo_results[0].boxes.cls #ID delle classi di appartenenza dell'oggetto

    #visualizza i risultati
    for i, (box, conf, cls_id) in enumerate(zip(detection, confidences, class_ids)):
        x1, y1, x2, y2 = map(int, box) #estrai le coordinate della bounding box
        label = f"{yolo_model.names[int(cls_id)]} ({conf:.2f})" #viene ncreata un'etichetta contenente il nome della classe (poiche class_ids e' un intero che grazie a model.names[cls_id] viene convertito nella corrispondente classe) e la confidenza

        #calcola la profondita' media nella bounding box
        depth_in_box = depth_map[y1:y2, x1:x2]
        average_depth = np.mean(depth_in_box) #media della profondita'

        detection_info.append({
            'frame': capture.get(cv2.CAP_PROP_POS_FRAMES),
            'label': label,
            'confidence': conf,
            'x1': x1,
            'x2': x2,
            'y1': y1,
            'y2': y2,
            'average_depth': average_depth
        })

        #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        #cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.9, (255, 0, 0), 2)
    return detection_info

    #mostra l'immagine con i bounding box
    #cv2.imshow("Detections", img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

In [ ]:
video_path = ""
capture = cv2.VideoCapture(video_path)
detection_info = []
while capture.isOpened():
    ret, frame = capture.read()
    if not ret:
        break #esci dal ciclo se non ci sono piu frame
    detection_info.append(detect_objects(frame))
capture.release()

In [11]:
detect_objects("./references/autostrada.jpg")


0: 448x640 17 cars, 2 trucks, 292.5ms
Speed: 3.0ms preprocess, 292.5ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 640)
